## Exploring the MPC's Isolated Tracklet File
#### Matthew J. Holman


12 November 2017

I am using this notebook as a starting point for the cs182 final project to link the ITF.

The plan is to:

    * Get the ITF records into a reasonable format, with the observatory locations determined, the time converted to jd_utc, jd_ut1, and jd_tdb, and the RA/Dec converted to unit vectors.
    * We need to keep the original format lines around, for later orbit fitting.
    * Separate the tracklets into months between successive full moons.
    * Determine the healpix region for each tracklet
    * Transform the tracklets in each time/healpix block into a local projection
    * Do the linear fit for each tracklet to get the a, b, g, adot, bdot solutions
    * Look for clusters in those solutions

### The NOVAS package

First, get the USNO's python NOVAS package.  We'll need that.

http://aa.usno.navy.mil/software/novas/novas_py/novaspy_intro.php

Just type 

pip install novas

pip install novas_de405

Here's the reference:

Barron, E. G., Kaplan, G. H., Bangert, J., Bartlett, J. L., Puatua, W., Harris, W., & Barrett, P. (2011) “Naval Observatory Vector Astrometry Software (NOVAS) Version 3.1, Introducing a Python Edition,” Bull. AAS, 43, 2011.

### The kepcart library

You will need to make sure you have a copy of the kepcart library.  There is a copy of it on the MPC bitbucket site, with some instructions.

In [1]:
%matplotlib inline
import numpy as np
import scipy.interpolate
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.notebook_repr_html', True)
import math
import kepcart as kc
import healpy as hp
import collections
import astropy

In [2]:
import MPC_library

In [3]:
Observatories = MPC_library.Observatories

In [4]:
ObservatoryXYZ = Observatories.ObservatoryXYZ

In [5]:
nside=32
for i in range(hp.nside2npix(nside)):
    print(i, hp.pix2vec(32, i, nest=True))

hp.query_disc(nside, (1.0, 0.0, 0.0), 0.000, inclusive=True, nest=True)

hp.vec2pix(nside, 1.0, 0.0, 0.0, nest=True)

0 (0.70695331253988136, 0.70695331253988125, 0.020833333333333332)
1 (0.68894172521921881, 0.72361812314290153, 0.041666666666666664)
2 (0.72361812314290175, 0.6889417252192187, 0.041666666666666664)
3 (0.70572436191476351, 0.7057243619147634, 0.0625)
4 (0.67024603285840201, 0.73950253916911868, 0.0625)
5 (0.65090093052950704, 0.75457506862563206, 0.083333333333333329)
6 (0.68714213557550174, 0.72172795503035236, 0.083333333333333329)
7 (0.66790557688419916, 0.73692024393589628, 0.10416666666666666)
8 (0.7395025391691189, 0.67024603285840179, 0.0625)
9 (0.72172795503035259, 0.68714213557550163, 0.083333333333333329)
10 (0.75457506862563228, 0.65090093052950682, 0.083333333333333329)
11 (0.7369202439358965, 0.66790557688419894, 0.10416666666666666)
12 (0.70326001790076054, 0.70326001790076043, 0.10416666666666666)
13 (0.68413230010135762, 0.71856662597008081, 0.125)
14 (0.71856662597008103, 0.6841323001013575, 0.125)
15 (0.69954722459920071, 0.6995472245992006, 0.14583333333333331)
16 (

4522

In [6]:
ObservatoryXYZ['W14']

(0.046526560882400647, -0.82059606538689689, 0.567774)

In [7]:
ObservatoryXYZ['261']

(-0.37785881193022558, -0.74609799883907901, 0.546877)

In [8]:
Observatories.getObservatoryPosition('F51', 2457000.5)

array([ 0.22807555,  0.87911034,  0.38114111])

In [9]:
Observatories.getObservatoryPosition('C51', 2457000.5)

(None, None, None)

### Reading the MPC data

Now let's look at some 80-character MPC records.

In [10]:
N = 10
with open('itf_new.txt', 'r') as f:
    lines = [next(f) for x in range(N)]
    for line in lines:
        print(line.rstrip('\n'))

     /07K07O* C1997 09 28.39921 01 17 21.49 +14 49 44.2          21.4 Vi     691
     /07K07O  C1997 09 28.42094 01 17 20.46 +14 49 38.3          21.1 Vi     691
     /07K07O  C1997 09 28.44271 01 17 19.44 +14 49 31.8          20.9 Vi     691
     /96T6P   C1997 09 08.46880 01 25 22.70 +14 41 09.1                      675
     /96T6P   C1997 09 08.51471 01 25 22.27 +14 41 01.1                      675
     /97P02Q  C1997 09 28.08296 21 18 43.02 -11 52 39.3          19.6 R      824
     /97P02Q  C1997 09 28.08750 21 18 43.01 -11 52 39.1          19.5 R      824
     /97P02Q  C1997 09 28.09190 21 18 43.01 -11 52 38.5          19.3 R      824
     /97QQ1   C1997 09 04.85836 20 50 49.29 -19 06 47.9          19.5 V      552
     /a8S27K* C1997 09 29.18338 23 00 56.39 -06 46 56.1          20.2 Vi     691


### Reading the MPC observation files

Dealing with files line by line in python is not fast.  

The itf.txt, NumObs.txt, and UnnObs.txt files have a mix of 1-line and 2-line formats.  


In [11]:
# This routine checks the 80-character input line to see if it contains a special character (S, R, or V) that indicates a 2-line 
# record.
def is_two_line(line):
    note2 = line[14]
    return note2=='S' or note2=='R' or note2=='V'

In [12]:
# This routine opens and reads filename, separating the records into those in the 1-line and 2-line formats.
# The 2-line format lines are merged into single 160-character records for processing line-by-line.
def split_MPC_file(filename):
    filename_1_line = filename.rstrip('.txt')+"_1_line.txt"
    filename_2_line = filename.rstrip('.txt')+"_2_line.txt"
    with open(filename_1_line, 'w') as f1_out, open(filename_2_line, 'w') as f2_out:
        line1=None
        with open(filename, 'r') as f:
            for line in f:
                if is_two_line(line):
                    line1=line
                    continue
                if line1 != None:
                    merged_lines = line1.rstrip('\n') + line
                    f2_out.write(merged_lines)
                    line1 = None
                else:
                    f1_out.write(line)
                    line1 = None


Now split the files for the three main MPC observation files.  (Need to convert the section below to code from markdown, if you haven't run it already.)

In [ ]:
split_MPC_file('itf_new.txt')

Below are routines that read the files after they have been split into their respective formats.  

In [13]:
def readMPC_1_line(filename='NumObs_1_line.txt', nrows=1000000):
    colspecs = [(0, 5), (5, 12), (12, 13), (13, 14), (14, 15), (15, 32), (32, 44), (44, 56), (65, 71), (77, 80)]
    colnames = ['objName', 'provDesig', 'disAst', 'note1', 'note2', 'dateObs', 'RA', 'Dec', 'MagFilter', 'obsCode']
    df = pd.read_fwf(filename, colspecs=colspecs, names=colnames, header=None, nrows=nrows)
    return df

In [14]:
def convertObs80(line):
    objName   = line[0:5]
    provDesig = line[5:12]
    disAst    = line[12:13]
    note1     = line[13:14]
    note2     = line[14:15]
    dateObs   = line[15:32]
    RA        = line[32:44]
    Dec       = line[44:56]
    mag       = line[65:70]
    filt      = line[70:71]
    obsCode   = line[77:80]
    return objName, provDesig, disAst, note1, note2, dateObs, RA, Dec, mag, filt, obsCode


In [15]:
def splitMagFilter(magFilter):
    pieces = magFilter.split()
    if len(pieces)==0:
        return None, None
    elif len(pieces)==1:
         return pieces[0], None
    else:
        return pieces[0], pieces[1]

        

In [16]:
N = 1000
with open('itf_new_1_line.txt', 'r') as f:
    lines = [next(f) for x in range(N)]
    for line in lines:
        objName, provDesig, disAst, note1, note2, dateObs, RA, Dec, mag, filt, obsCode = convertObs80(line)
        jd_utc = MPC_library.date2JD(dateObs)
        jd_tdb  = MPC_library.EOP.jdTDB(jd_utc)
        xh, yh, zh = Observatories.getObservatoryPosition(obsCode, jd_utc)
        #mag, filt = splitMagFilter(magFilter)
        print("%7s %13.6lf %13.6lf %11.6lf %11.6lf %11.6lf %11.6lf %11.6lf %s %s"% \
              (provDesig, jd_utc, jd_tdb, MPC_library.RA2degRA(RA), MPC_library.Dec2degDec(Dec), xh, yh, zh, mag, filt))



/07K07O 2450719.899210 2450719.899941   19.339542   14.828944    0.997647    0.085482    0.037077 21.4  V
/07K07O 2450719.920940 2450719.921671   19.335250   14.827306    0.997603    0.085826    0.037224 21.1  V
/07K07O 2450719.942710 2450719.943441   19.331000   14.825500    0.997558    0.086170    0.037372 20.9  V
/96T6P  2450699.968800 2450699.969531   21.344583   14.685861    0.976969   -0.225464   -0.097741        
/96T6P  2450700.014710 2450700.015441   21.342792   14.683639    0.977141   -0.224757   -0.097437        
/97P02Q 2450719.582960 2450719.583691  319.679250  -11.877583    0.998222    0.080486    0.034936 19.6  R
/97P02Q 2450719.587500 2450719.588231  319.679208  -11.877528    0.998215    0.080558    0.034967 19.5  R
/97P02Q 2450719.591900 2450719.592631  319.679208  -11.877361    0.998207    0.080627    0.034997 19.3  R
/97QQ1  2450696.358360 2450696.359091  312.705375  -19.113306    0.960997   -0.280126   -0.121411 19.5  V
/a8S27K 2450720.683380 2450720.684111  345.234

In [17]:
with open('itf_new_1_line.mpc', 'w') as outfile:
    with open('itf_new_1_line.txt', 'r') as f:
        for line in f:
            objName, provDesig, disAst, note1, note2, dateObs, RA, Dec, mag, filt, obsCode = convertObs80(line)
            jd_tdb  = MPC_library.EOP.jdTDB(jd_utc)
            jd_utc = MPC_library.date2JD(dateObs)
            xh, yh, zh = Observatories.getObservatoryPosition(obsCode, jd_utc)
            outstring = "%7s %13.6lf %13.6lf %11.6lf %11.6lf %11.6lf %11.6lf %11.6lf %s %s\n"% \
                  (provDesig, jd_utc, jd_tdb, MPC_library.RA2degRA(RA), MPC_library.Dec2degDec(Dec), xh, yh, zh, mag, filt)
            outfile.write(outstring)




ValueError: A value in x_new is below the interpolation range.

In [ ]:
def readMPC_2_line(filename='NumObs_2_line.txt', nrows=1000000):
    colspecs = [(0, 5), (5, 12), (12, 13), (13, 14), (14, 15), (15, 32), (32, 44), (44, 56), (65, 71), (77, 80), (112,113), (114,149)]
    colnames = ['objName', 'provDesig', 'disAst', 'note1', 'note2', 'dateObs', 'RA', 'Dec', 'MagFilter', 'obsCode', 'units', 'xyz']
    df = pd.read_fwf(filename, colspecs=colspecs, names=colnames, header=None, nrows=nrows)
    return df

### Looking at the Isolated Tracklet File (ITF)

We will start with the isolated tracklet file (ITF).  The ITF had 10.8 million lines, as of Dec 2015.  

* G96 accounts for 4.13 million. [4.44M]
* F51 accounts for 3.85 million. [4.63M]
* 691 accounts for 1.36 million.
* 703 accounts for 0.33 million.
* 644 accounts for 0.16 million.
* 291 accounts for 0.13 million.
* 699 accounts for 0.11 million.
* 807 accounts for 0.10 million

The ITF now has 11.9 million lines.

In [ ]:
4.13 + 3.85 + 1.36 + 0.33 + 0.16 + 0.13 + 0.11 + 0.10

### The 1-line ITF entries

First, let's read and process the 1-line ITF entries.  We will add a few relevant columns.

In [ ]:
%%time
df = readMPC_1_line('itf_1_line.txt', nrows=100000000)

In [ ]:
df

readMPC_1_line is slow because it involves file I/O.

In [ ]:
%%time
df['JD_utc'] = df['dateObs'].apply(date2JD)

The call above is slow because it involves repeated, single calls to the novas julian_date function.  It could be made much faster by using a routine that will do many calculations on the c-side all at once.

In [ ]:
%%time
df['RA_deg'] = df['RA'].apply(RA2degRA)
df['Dec_deg'] = df['Dec'].apply(Dec2degDec)

This whole conversion is a fairly slow process, even for just 10 million lines.  It takes about 4 minutes in total, so it might take an hour for 130 million lines.  That's much, much too slow.  

### The 2-line ITF entries

Next, let's read and process the 2-line ITF entries.

In [ ]:
%%time
df2 = readMPC_2_line('itf_2_line.txt')
df2['JD_utc'] = df2['dateObs'].apply(date2JD)
df2['RA_deg'] = df2['RA'].apply(RA2degRA)
df2['Dec_deg'] = df2['Dec'].apply(Dec2degDec)

In [ ]:
# This routine parses the section of the second line that encodes the geocentric satellite position.
def parseXYZ(xyz):
    xs = xyz[0]
    x = float(xyz[1:11])
    if xs=='-':
        x = -x
    ys = xyz[12]
    y = float(xyz[13:23])
    if ys=='-':
        y = -y
    zs = xyz[24]
    z = float(xyz[25:])
    if zs=='-':
        z = -z
    return x, y, z

In [ ]:
def getSatellitePosition(xyz, dateObs):
    x,y,z = parseXYZ(xyz)
    x /= au_km
    y /= au_km
    z /= au_km
    jd_utc = date2JD(dateObs)
    leaps = getLeapSeconds(jd_utc)
    jd_tt = jd_utc + (32.184 + leaps)/(24.0*60*60)
    pos = getEarthPosition(jd_tt) # Might need to switch to tdb
    heliocentric_vec = pos[0]+x, pos[1]+y, pos[2]+z
    return heliocentric_vec


In [ ]:
# Now calculate the heliocentric satellite positions and add to a dataframe.
position_array2 = np.array(list(map(lambda xyz, dateObs: getSatellitePosition(xyz, dateObs), df2['xyz'], df2['dateObs'])))
position_df2 = pd.DataFrame(position_array2, columns=['xh', 'yh', 'zh'])
result2 = pd.concat([df2, position_df2], axis=1, join_axes=[df2.index])
result2=result2.drop(['xyz'], axis=1)
result2=result2.drop(['units'], axis=1)

### Merge the 1-line and 2-line ITF results

In [ ]:
result3 = result.append(result2, ignore_index=True)
newdf = result3[result3.obsCode != '247']  # There were a few errant code 247 entries in the ITF I am looking at.

Now drop some of the intermediate, but large, tables.

In [ ]:
df = None
result = None
result2 = None
result3 = None
position_df = None
position_df2 = None

In [ ]:
%%time
jd_tts = np.linspace(jd_tt, jd_tt+1000, 1000)
positions, velocities = [], []
for jd_tt in jd_tts:
    pos, vel = solsys.solarsystem(jd_tt, 3, 1)
    positions.append(pos)
    velocities.append(vel)

Let's drop some columns that will not be used in their original form.

In [ ]:
newdf['disc'] = (newdf.disAst=='*')
newdf=newdf.drop(['dateObs','RA','Dec', 'note1', 'note2', 'disAst'], axis=1)

There are a few objects with names that do not belong in the ITF.  Let's drop those, but this is not a general function.  I don't think there are any in the current version.


In [ ]:
newdf = newdf[newdf.objName.isnull()]

In [ ]:
newdf.head()

### Dealing with MPC orbits

Ok, now that we have the ITF in a pandas dataframe, it's time to work with MPC orbits.  I am going to do this in a way that takes a few steps toward a new approach for MPChecker.  

I wrote a little library in c that will rapidly compute the cartesian states (positions + velocities) given input arrays of orbital elemnets.  I need to make the accessible too.

In [ ]:
%%time
# Here I am just checking the speed of the external cartesians routine.
GMsun = 2.9591220828559115e-04 
num = 1000000
a_arr = np.ones((num))
e_arr = np.ones(num)*0.1
incl_arr= np.full((num), 0.1)
longnode_arr = np.full((num), 0.2)
argperi_arr = np.full((num), 0.1)
meananom_arr = np.linspace(0.0, 2.0*np.pi, num)

state_arr = kc.cartesians(num, GMsun, a_arr, e_arr, incl_arr, longnode_arr, argperi_arr, meananom_arr)

In [ ]:
kc.keplerians(num, GMsun, state_arr)

In [ ]:
elts = np.reshape(np.stack([a_arr, e_arr, incl_arr, longnode_arr, argperi_arr, meananom_arr]).T, -1)
elts.shape

In [ ]:
%%time
ps, vs = kc.cartesian_elements(num, GMsun, elts)

In [ ]:
ps[0:3]

In [ ]:
np.sqrt(GMsun/(700.*700.*700.))

In [ ]:
for i, st in enumerate(state_arr):
    if i%1000000==0:
        print(i, st.x, st.y, st.z)

Now that there are functions that can rapidly transform sets of orbital elements and dynamical states, it should be possible to write the functionality of observe_minor_planet in python.  DONE.

compute_position takes a set of orbital elements, a reference epoch, and a specified time and then determines the heliocentric and geocentric positions of the object at the specfied time, corrected for light time.

Let's start by reading an arbitrary file of MPC orbital elements.

In [ ]:
orb_df.desig.unique().shape

In [ ]:
def getStates(orb_df, jd_tt):
    meananoms = orb_df.meananom.values + orb_df.n*(jd_tt-orb_df.epoch_jd_tt)
    sts = kc.cartesians(orb_df.shape[0], GMsun, orb_df.a.values, orb_df.e.values, orb_df.incl.values, orb_df.longnode.values, orb_df.argperi.values, meananoms.values)
    return sts

In [ ]:
%%time
sts=getStates(orb_df, 2457371.5)

In [ ]:
def getPositionsVelocities(orbs_df, jd_tt, lt):
    meananoms = orbs_df.meananom.values + orbs_df.n.values*(jd_tt-lt-orbs_df.epoch_jd_tt.values)
    positions, velocities = kc.cartesian_vectors(orbs_df.shape[0], GMsun, orbs_df.a.values, orbs_df.e.values, orbs_df.incl.values, orbs_df.longnode.values, orbs_df.argperi.values, meananoms)
    return np.array(positions).reshape((-1,3)), np.array(velocities).reshape((-1,3))

In [ ]:
%%time
positions, velocities = getPositionsVelocities(orb_df, 2457371.5, 0.0)

In [ ]:
f = np.dot(positions, rotate_matrix(ecl).T)[0]
g = novas.ecl2equ_vec(2457371.5, positions[0])
f-g

In [ ]:
# This routine gets the geocentric position vectors of all
# the minor planets, corrected for light time.  It assumes
# that 3 iterations is adequate.  That assumption needs to 
# be checked.
speed_of_light = 2.99792458e5 * 86400./au_km
rot_mat = rotate_matrix(ecl)
def getGeocentricPositions(orb_df, jd_utc, obsCode=None):
    jd_tt = jdTT(jd_utc)
    # Assume that the light time is 20 minutes as a first guess.    
    lt = np.ones(orb_df.shape[0])*(20./(24.*60.))
    if obsCode==None:
        pos = getEarthPosition(jd_tt) # Might need to switch to tdb, and this really needs to be the observatory position
    else:
        pos = getObservatoryPos(obsCode, jd_utc)
    for i in range(3):
        positions, _ = getPositionsVelocities(orb_df, jd_tt, lt)
        eq_positions = np.dot(rot_mat, positions.T).T
        geopos = eq_positions-pos
        distances = np.linalg.norm(geopos, axis=1)
        lt = distances/speed_of_light
    return geopos, distances

I might need a version of getGeocentricPositions that takes a set of elements and a set of times and returns a set of positions.

In [ ]:
%%time
geopos, distances = getGeocentricPositions(orb_df, 2457360.5, '500')

In [ ]:
for t in np.arange(2457370.5, 2457380.5, 1.):
    geopos, distances = getGeocentricPositions(orb_df, t, 'F51')
    geopos_normed=geopos/distances[:, np.newaxis]
    ra, dec = novas.vector2radec(tuple(geopos[1]))
    print(t, 15.0*ra, dec)

### Read all the MPC orbit files

In [ ]:
# Generate a list of mpcorb files.  These contain osculating orbital elements
# every 40 days. For now I just copied some files I got from Gareth for Pan-STARRS-1
# work.  However, I will soon be able to generate my own.
from os import listdir
from os.path import isfile, join
mypath = 'elements/mpcorb'
mpc_files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f)) & f.endswith('.txt')]

epochs = [f.rstrip('.txt').lstrip('mpcorb_') for f in listdir(mypath) if isfile(join(mypath, f)) & f.endswith('.txt')]
epochs_jd = np.array(list(map(yrmndy2JD, map(convertEpoch, epochs))))

## This function identifies the orbit file with the epoch that is nearest in time 
## to the given date.
##
## This function is using two global variables.  That needs to be changed.
## They are epochs_jd and mpc_files.
###
def getMPCorbFile(jd_tt):
    spacing = (epochs_jd[-1] - epochs_jd[0])/(len(epochs_jd)-1)
    max_idx = epochs_jd.shape[0]-1
    idx = np.searchsorted(epochs_jd, jd_tt+spacing/2.0, side='right')-1
    if idx < 0:
        return mpc_files[0]
    if idx > max_idx:
        return mpc_files[max_idx]
    else:
        return mpc_files[idx]
   

In [ ]:
 ## Now build a dictionary of all the orbit dataframes.
orb_df_dict={}
for i, (epoch, mpc_file) in enumerate(zip(epochs, mpc_files)):
    print(i, epoch, mpc_file)
    orb_df_dict[epoch] = readMPCorbits(filename=mpc_file)

In [ ]:
len(orb_df_dict)

### Trying to match objects in the ITF with those in the MPC orbits catalog.

The plan here is to use the sets of keplerian orbits of MPC objects to predict their sky plane positions at the times of a observations in the ITF to see if anything matches up.  There are several steps.  

The brute force way would be to predict the positions of ALL the objects at ALL of the unique times of observation in the ITF.  That would be very slow.  This is the time to take a few more steps towards a faster MPChecker.

The plan is to predict the geocentric position vectors, corrected for light time, of all the objects in the MPC orbit catalog on intervals of 1 day.  I will then organize the results in a way that can be used efficiently to check which objects might match observations in the ITF.

For each time in the range of times:

* Run getGeocentricPositions
* Determine the HEALPix numbers for all of those positions
* Organize the results for rapid searching.

[NOTE: the HDF5 approach seems too slow for now.]  There can be an HDF5 directory structure that has MJD as the first division (there might be 1e3 days).  The next division would be HEALPix number.  There might be ~12K of those (with nside=32).  That's 1e3*1.2e4=1.2e7 directories.
There are 5.6e5 objects in the MPC orbit catalog.  That would be an average of 50 objects in each bin, but the polar bins would be largely empty and those on the ecliptic would be more full.  



Now I need to build the functions for querying the data.  Given a time, an RA/Dec position, and a search radius, find which minor planets are nearby.

* Given the time, find the nearest night or the two neighboring nights.  DONE.
* Given the RA/Dec position and the search radius, find the collection of healpix numbers that are covered by the search.  DONE.
* With the night (or nights) and healpix numbers in hand, query the HDF5 data structure for the minor planets that need to be considered.  (Actually, the HDF5 structure seems too slow.  I'm using a daily dictionary in memory.)
* Given the minor planets, find their elements and figure out the objects' positions.  This part needs to be faster, but it's ok for now.
* As the orbit files are updated, the data structures that support MPChecker need to also be updated automatically.
* The routine that computes geocentric and topocentric positions could be updated to be something that uses Chebyshev polynomials.  This holds promise for speeding up the process.


In [ ]:
%%time

currentFile = None
nside = 32
nested_scheme = True
pixels_dict={}
orb_df_dict={}

for i, jd_utc in enumerate(np.linspace(2456323., 2456423., 101)):
    orb_file = getMPCorbFile(jd_utc)
    if orb_file != currentFile:
        orb_df=readMPCorbits(orb_file)
        orb_df_dict[orb_file]=orb_df
        currentFile = orb_file
    geopos, distances = getGeocentricPositions(orb_df, jd_utc)
    geopos_normed=geopos/distances[:, np.newaxis]
    x, y, z = geopos_normed[:,0], geopos_normed[:,1], geopos_normed[:,2]
    pixels= hp.vec2pix(nside, x, y, z, nested_scheme)
    pixels_dict[jd_utc]=(orb_df.desig.values, pixels)
    

In [ ]:
final_dict={}
for jd, (desigs, pixels) in pixels_dict.items():
    #print jd
    test_dict={}
    test_dict=collections.defaultdict(list)
    for desig, pixel in zip(desigs, pixels):
        test_dict[pixel].append(desig)
    final_dict[jd]=test_dict


In [ ]:
#nside = 32
nside = 32768
nested_scheme = True
def ra_dec2pix(ra, dec, nside, nested=nested_scheme):
    phi   = ra*math.pi/180.
    theta = math.pi/2.0 - dec*math.pi/180.
    pix = hp.ang2pix(nside, theta, phi, nested)
    return pix

In [ ]:
for desig, ra, dec in zip(df.provDesig, df.RA_deg, df.Dec_deg):
    print(desig, ra,dec, ra_dec2pix(ra, dec, nside=nside, nested=True))

In [ ]:
for desig, ra, dec in zip(df.provDesig, df.RA_deg, df.Dec_deg):
    print(desig, ra,dec, ra_dec2pix(ra, dec, 32768, nested=True))

In [ ]:
df

In [ ]:
def get_orb_df(jd_utc):
    orb_file = getMPCorbFile(jd_utc)
    return orb_df_dict[orb_file]

In [ ]:
def MPChecker(jd_utc, ra, dec, sr, nside=32, nested=True):
    
    # final_dict is a global here.  It should be passed in
    # instead.
    
    # Adjust the date to one of the dictionary values
    jd_round = round(jd_utc)
    
    # Find the healpix regions covered by the search
    # Probably should expand the search radius to accommodate 
    # daily motion.
    phi   = ra*math.pi/180.
    theta = math.pi/2.0 - dec*math.pi/180.
    if sr<1.0:
        search_radius = 1.0*math.pi/180.
    else:
        search_radius = sr*math.pi/180.
    vec = hp.ang2vec(theta, phi)
    res = hp.query_disc(nside, vec, search_radius, nest=nested, inclusive=True)
    
    # Find the unique set of minor planets in the healpix regions
    mp_set = set()
    for pix in res:
        mps = final_dict[jd_round][pix]
        '''
        fname = "%.1lf" % (jd_floor)
        mps = list(f[fname][str(pix)])
        '''
        mp_set.update(mps)

    # This section, which does an "isin" call, is the
    # bottleneck.  It could be faster if final_dict contained
    # not only the minor planet IDs but also their elements.
    # That might require too much space.  Perhaps the "get_orb_df"
    # and sub_orb_df calls could be combined.
    #
    # Compute the positions of those minor planets
    orb_df = get_orb_df(jd_utc)
    sub_orb_df = orb_df[orb_df['desig'].isin(mp_set)]
    geopos, distances = getGeocentricPositions(sub_orb_df, jd_utc)
    
    # Now determine which MPs are actually in the search region
    geopos_normed=geopos/distances[:, np.newaxis]
    angles = np.degrees(np.arccos(np.dot(geopos_normed, vec)))
    
    return sub_orb_df.desig[angles<sr], geopos[angles<sr], distances[angles<sr]

In [ ]:
%time desigs, geopos, distances = MPChecker(2456374.0, 30, -0, 0.5)

MPChecker returns:
designation
distance (RA/Dec)
magnitude

Seems to be duplication in here

In [ ]:
desigs, geopos, distances

In [ ]:
4/65e-3

In [ ]:
count=0
for k, v in final_dict[2456323.0].items():
    print(k, len(v))
    count += len(v)
print(count)

In [ ]:
len(desigs)

In [ ]:
test_df = orb_df.set_index('desig')

In [ ]:
desigs

In [ ]:
test_df

In [ ]:
%%time
test_df2 = test_df.ix[desigs]

In [ ]:
%%time
test_df3 = orb_df[orb_df.desig.isin(desigs)]

In [ ]:
orb_df.info()

In [ ]:
for desig, gp in zip(desigs, geopos):
    ra, dec = novas.vector2radec(tuple(gp))
    print("%10s %10.5lf %10.5lf" % (desig, ra, dec))

In [ ]:
%%time
orb_df.iloc[range(0,6000, 10)]

In [ ]:
counts = [(item, count) for item, count in collections.Counter(orb_df.desig).items() if count > 1]

In [ ]:
len(counts)

In [ ]:
len(final_dict[2456323.0])

In [ ]:
pixels_dict[2456323.0]

### Next steps on matching on objects with observations in the ITF.

Now that I have a working version of MPChecker, as well as "final_dict", a dictionary keyed on jd_utc at one day intervals that has as values other dictionaries that have healpix numbers as keys and lists of minor planets as values, I should be able to match observations in the ITF.

Basically, I know where all the minor planets with orbits are on a daily basis.  How can I use that efficiently to check for matches?  

There are 1.7e6 unique times in the ITF.  There are 3-4 times per tracklet, so perhaps there are 5e5 unique tracklet start times.  And maybe only half of those are in the past few years: 2.5e5 times or 250 per night.

Sort the tracklets by time and use the results to determine the approximate exposure centers and FOV radii for each observatory.  Then run MPChecker on those pointing histories.  Each run will result in ~1000 minor planets, depending upon the FOV and the sky location.

Is it feasible?  2.5e5 times * 80e-3 sec/time = 5.6 hours, so yes.

### The MPCORB.DAT file

The MPCORB.DAT file has three sections.

* First, the numbered objects with the most well-determined orbits.

* Second, the unnumbered objects with reasonably good orbits that include perturbations.

* Third, the unnumbered objects with fairly short arcs that have unperturbed orbits.

The first group can definitely go into a system like Chebyshev polynomials.  The second group probably can too.  The third group needs to be dealt with separately; their epochs are all different.

### Connecting the dots

Despite the differences in arc length and orbit quality, every object in the MPCORB.DAT file is an orbit.  Each of these would represent a good starting point for a more refined orbit.


In [ ]:
3e5*80e-3/3600

In [ ]:
distant_Num = np.sort(orb_df[(orb_df.a > 30) & (orb_df.desig.apply(len)<=5)].desig)
distant_Unn = np.sort(orb_df[(orb_df.a > 30) & (orb_df.desig.apply(len)>5)].desig)

In [ ]:
len(distant_Num), len(distant_Unn)

In [ ]:
split_MPC_file('NumObs.txt')

In [ ]:
distant_Num_file = open('distant_num.txt', 'w')
with open('NumObs_1_line.txt') as data_file:
    first=True
    # read past the header
    data_line = data_file.readline()
    data_name = data_line[0:5]
    count = 0
    for desig in distant_Num:
        print(desig)
        while(data_name < desig and data_line !=''):
            data_line = data_file.readline()
            data_name = data_line[0:5]
            count += 1
        while(data_name == desig and data_line !=''):
            obscode = data_line[77:80]
            distant_Num_file.write(data_line)
            data_line = data_file.readline()
            data_name = data_line[0:5]
            count += 1
        
distant_Num_file.close()
        

In [ ]:
distant_Unn_file = open('distant_unn.txt', 'w')
with open('UnnObs_1_line.txt') as data_file:
    first=True
    # read past the header
    data_line = data_file.readline()
    data_name = data_line[5:12]
    count = 0
    for desig in distant_Unn:
        print(desig)
        while(data_name < desig and data_line !=''):
            data_line = data_file.readline()
            data_name = data_line[5:12]
            count += 1
        while(data_name == desig and data_line !=''):
            distant_Unn_file.write(data_line)
            data_line = data_file.readline()
            data_name = data_line[5:12]
            count += 1
        
distant_Unn_file.close()
        

In [ ]:
-15+ -7/60. - 22.1/3600.

In [ ]:
RA2degRA('17 06 14.318')

In [ ]:
Dec2degDec('-15 23 39.75')

In [ ]:
RA2degRA('17 06 42.297'), 15*(17 + 6/60. + 42.297/3600.)


### Using Chebyshev polynomials

It's possible to compute the x,y,z cartesian positions of all the minor planets as a function of time, either by using the orbital elements or by numerically integrating their orbits, and to then compute sets of Chebyshev polynomial coefficients that will allow for very rapid interpolation of those positions.  This is basically the approach used with the JPL ephemerides.  Now we would be using the same approach for minor planets, but the level of precision needed is much courser.

It would also be possible to compute and record the geocentric light time so that it would be immediately available.  The relative precision on the light time would not need to be that great.  The light time might typically be 200 sec; we might not need much better than 1 part in 1e4 or 1e5 on that, since this is meant to be a quick but rough tool.

* Take jd_utc and compute jd_tt
* Given the RA/Dec and search radius, grab the healpix regions
* Grab all the minor planets in those healpix regions
* Get the light time for those objects
* Interpolate the x,y,z positions of those minor planets at the corresponding values of jd_tt-light_time

I am guessing that 1000 calculations of this would take 100 microseconds.

If that is the case, then 1e7 calculations would take 1e7/1000 * 100e-6 sec = 0.1 sec!

We can ignore the observatory position for the light time calculation.
7000km/3AU = 7000/4.5e8 = 1.5e-5

7000km/3e5km/s

### Integrating the orbits

Here I am trying to do to a reasonable first job on integrating the orbits of minor planets, starting with the most recent version of MPCORB.DAT.

Gareth told me that he integrates the orbits of the minor planets in the field of the sun, planets, and the biggest few asteriods in a "one pass process."  I believe that he integrates the orbits of the biggest few asteroids in the field of the sun and planets.  At the same time he integrates the orbits of the other asteroids in the field of the sun, planets, and biggest few asteroids.

What I think I want to do is integrate the orbits of the biggest few asteroids in the field of each other, the sun, and the planets.  And, at the same time, I integrate the orbits of the other asteorids in the field of all the masses.  It probably doesn't make a difference, but I'll try it.

Just to get started quickly, I will take the positions of the sun and planets from DE405, which I have already.  I will store those on one day intervals.  I will also check to see if I can get the barycentric acceleration of the sun from DE405.  

Then I will gather the orbital elements of all the minor planets from the MPCORB.DAT file at its epoch.  I will associate masses with some of these bodies and take the rest of the bodies as test particles.

I need a slightly modified version of an n-body map that allows the positions (and velocities) of the sun and the planets to come from a file but then allows some additional masses that interact with each other but do not affect the sun and planets.

Once I have that, so that I can integrate the orbits of the minor planets with a fixed time step, I then need to be able to integrate to specific times.  For that I will grab one of the simple symplectic correctors, which should do an adequate job.

Then I need to determine the positions of the minor planets at the times of the Chebyshev nodes.  That will involve integrating the orbits to the specific times of the Chebyshev nodes.  An alternative is to integrate a uniform spacing of times and then fit for the Chebysev coefficients to a given order.  That would be easier.

### Exploring Chebyshev interpolation

Fortunately, some facility already exists in numpy.

In [ ]:
np.polynomial.chebyshev.chebfit

This is trying to reproduce some results I saw on the web.  I liked this link because it's short and clear, but there are many: http://www.math.wsu.edu/faculty/genz/448/lessons/l303.pdf

Here I am making a test data set, with uniformly spaced samples of a portion of a sine function, with 25 points.  Then I fit the Chebyshev coefficients.  The function `chebfit` does a least squares fit, because the data are not actually sampled at the Chebyshev nodes.  I try a lower order polynomial, 10 in this case.


Then take that Chebyshev function and evaluate it with much finer sampling, and plot the difference.  The differences are really quite small, and they vanish quickly as the Chebyshev order is increased.   The differnces are 1e-6 with n=5 and 1e-13 for n=10.  For n=13, we reach the limit of double precision. Of course, the sine is a very smooth function, but so are the cartesian positions of solar system bodies.



In [ ]:
npts=25
x = np.linspace(0.0, np.pi*0.4, npts)
y = np.sin(x)
y2 = np.cos(x)
yarr = np.column_stack([y, y2, y2])

In [ ]:
%%time
c = np.polynomial.chebyshev.chebfit(x, yarr, 13)

Suppose it takes 0.8 ms to do one x,y,z fit for a segment.  7e5 objects, with ~100 segments, means 5e7 ms or 5e4 sec or 12 hours.  Seems long.  Of course, this could be split over several machines.



In [ ]:
7e4/3600.

In [ ]:
npts=1000
x = np.linspace(0.0, np.pi*0.4, npts)
y = np.sin(x)
y2 = np.cos(x)
t = x
pt = np.polynomial.chebyshev.chebval(t, c)
diff = pt[1,:]-y2

#plt.plot(x, y3, 'r.')   
#plt.plot(t, pt, 'k-', lw=3) 
plt.plot(t, diff)

In [ ]:
%%time
pt = np.polynomial.chebyshev.chebval(t, c)

In [ ]:
c.shape, x.shape, pt.T.shape

In [ ]:
npts=50
x = np.linspace(0.0, np.pi*0.5, npts)
y = np.sin(x) 
c = np.polynomial.chebyshev.chebfit(x, y, 13)
dc = np.polynomial.chebyshev.chebder(c)

npts=1000
x = np.linspace(0.0, np.pi*0.5, npts)
y = np.cos(x) 
t = x
pt = np.polynomial.chebyshev.chebval(t, dc)
diff = pt-y

#plt.plot(x, y, 'r.')   
#plt.plot(t, pt, 'k-', lw=3) 
plt.plot(t, diff)


So, this is very promising.  With a fairly small number of coefficients (7-13) we can represent the data with the precision we want.  A modest number of uniformly spaced samples seems to do the job.  Perhaps twice as many samples as the polynomial order is about right.  The time interval should be a fraction of an orbital period.

The questions are:

* Do we want to represent the full precision?

* Do we want to represent just positions or also velocities?  If we can represent the postions to machine precision, we can the velocities from numerical differentiation.  Although they might be a factor of a few less precise, that's ok.

Based on the simple experiemnts above, the Chebyshev derivative is certainly accurate enough.  So, we should go for the full precision but keep just the coefficients for the positions.  


### Returning to NOVAS in C.  

I was finally able to get the fortran compiler to work on my Mac laptop again after the hard drive failure.  So it is worth exploring what elements of what I have developed here in python would be better in C or C++.

Here are a few thoughts:

* I need to edit kepcart so that an array of states is really just an array of positions and an array of velocities.  That will make the results easier to work with.  DONE.

* I could alter the orbital element input similarly, so that it's just an array of elements rather than separate arrays for ach element.

* I need to think about carrying a vector of the light time values for each of the bodies as a parameter to getStates.  That way getStates can be easily called in an iteration.  DONE.

* An alternative is to create the function entirely on the c-side, such that I pass in the position of the observatory as a parameter.



In [ ]:
newdf.JD_utc.min(), newdf.JD_utc.max()

In [ ]:
len(newdf.JD_utc.unique())/1e6

In [ ]:
n, nbins, patches = plt.hist(newdf.JD_utc, bins=100, range=[2450000, 2457500])
plt.xlabel("JD")
plt.ylabel("Number of exposures")



Indeed, the bulk of the observations are within the last 1,000 days.

### Fiddling around with HDF5

In [ ]:
hdfn= getMPCorbFile(2457371.50).rsplit('.txt')[0]+'.hdf5'

orb_df2 = pd.read_hdf(hdfn)

orb_df2[0:2]

jd_tt = 2457360.5
meananoms = orb_df2.meananom.values + orb_df2.n*(jd_tt-orb_df2.epoch_jd_tt)
sts = kc.cartesians(orb_df2.shape[0], GMsun, orb_df2.a.values, orb_df2.e.values, orb_df2.incl.values, orb_df2.longnode.values, orb_df2.argperi.values, meananoms.values)


orb_df[0:2]

orb_df_m = pd.merge(orb_df, orb_df2, how='inner')

orb_df_m.tail()

In [ ]:
f = h5py.File('MPChecker.hd5', 'w')
for jd, (desigs, pixels) in pixels_dict.iteritems():
    print jd
    test_dict={}
    test_dict=collections.defaultdict(list)
    for desig, pixel in zip(desigs, pixels):
        test_dict[pixel].append(desig)
    for pixel, mp_array in test_dict.iteritems():
        fname= '%.1lf/%d' % (jd, pixel)
        dset = f.create_dataset(fname, data=mp_array)

f.close()

In [ ]:
f = h5py.File('MPChecker.hd5', 'r')

if '2456423.0/1000' in f:
    for v in f['2456423.0/1000']:
        print v

In [ ]:
f[str(2456323.0)][str(4543)]

In [ ]:
for t in f:
    print t, 'K05UE9M' in f[t]['151']


In [ ]:
for i, (epoch, mpc_file) in enumerate(zip(epochs, mpc_files)):
    hdfn = mpc_file.rsplit('.txt')[0]+'.hdf5'
    orb_df_dict[epoch].to_hdf(hdfn, 'orb_df')    
    print i, epoch, mpc_file

### Working with getting sun and planet positions for numerical integrations

In [ ]:
solsys.solarsystem(jd_tt, 3, 0)

In [ ]:
eph_manager.planet_ephemeris((jd_tt, 0.0), 2, 11)

In [ ]:
eph_manager.planet_ephemeris((jd_tt, 0.0), 10, 11)

In [ ]:
solsys.solarsystem(jd_tt, 11, 0)

In [ ]:
%%time
for jd_tt in np.linspace(jd_tt, jd_tt+1000, 11):
    for body in range(0,11):
        pos, vel = eph_manager.planet_ephemeris((jd_tt, 0.0), body, 11)
        print(jd_tt, body, pos)

In [ ]:
# This routine writes the barycentric positions of the planets, moon, and sun, from DE405, for a sequence of
# days to the named file.
def write_positions(filename, jd_start, jd_end, ndates):
    with open(filename, 'w') as f:
        for jd in np.linspace(jd_start, jd_end, ndates):
            f.write(str(jd) + "\n")
            for body in range(0, 11):
                (x, y, z), (vx, vy, vz) = eph_manager.planet_ephemeris((jd, 0.0), body, 11)
                f.write(str(body) + " " + str(x) + " " + str(y) + " " + str(z) + " " + str(vx) + " " + str(vy) + " " + str(vz) + "\n")



In [ ]:
yrmndy2JD(convertEpoch('K161D'))

In [ ]:
jd_start = 2457400.5
write_positions("foo.out.3", jd_start, jd_start-10000, 40001)

In [ ]:
import novas.constants

In [ ]:
novas.constants.C_AUDAY

In [ ]:
GMsun = novas.constants.GS*(np.power(86400, 2.0)/np.power(novas.constants.AU, 3.0))
GMsun

In [ ]:
bodies = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto', 'Sun', 'Moon']
print("%s %.15le" % ("GMsun ", GMsun))
for i, body in enumerate(bodies):
    print(i, body, novas.constants.RMASS[body])

In [ ]:
GM = 1.32712440018e20

In [ ]:
au = 149597870700.
dy = 86400.

In [ ]:
GM*(1/(au*au*au))*(dy*dy)/1000

In [ ]:
5e-6*3600

In [ ]:
6000/15